In [1]:
import pandas as pd
import os
import shutil
import sqlite3

In [3]:
landing = '/workspaces/api-geral/GEO/CEP/dados/landing/'

leitor_landing = os.listdir(landing)
print(leitor_landing)

['resultado_concat_20240501_162447.txt']


In [36]:
from datetime import datetime
import os.path


timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

bckp_path = '/workspaces/api-geral/GEO/CEP/ingestion/dados_bckp/'
landing = '/workspaces/api-geral/GEO/CEP/dados/landing/'
word_key = 'resultado'

ultimo_timestamp = None
caminho_arquivo = None


for root, dirs, files in os.walk(landing):
    for nome_arquivo in files:
        if word_key in nome_arquivo:
            caminho_completo = os.path.join(root, nome_arquivo)
            timestamp = os.path.getmtime(caminho_completo)
            if ultimo_timestamp is None or timestamp > ultimo_timestamp:
                ultimo_timestamp = timestamp
                caminho_arquivo = caminho_completo
                checking = os.path.isfile(caminho_arquivo)
                if checking == True:
                    print(checking)
                else:
                    print('Error checking')
                if os.path.isfile(caminho_arquivo):
                    file_extension = os.path.splitext(caminho_arquivo)[1]
                    if file_extension.lower() in {'.txt', '.csv'}:
                        print(f'file extension csv or txt, ok! extension: {file_extension}')
                    elif file_extension.lower() in {'.bin'}:
                        print('file extension binary, ok')
                        print(file_extension)
                    else:
                        print('extension error, not found')

if caminho_arquivo:
    df = pd.read_csv(caminho_arquivo, sep=';')
    file_bckp = f'{caminho_arquivo}'
    df_ccep = df['cep'].str.replace(r'-', '')
    df_cep =df['cep'].str.extract(r'(\d{5})-\d{3}')
    df_cep_compl = df['cep'].str.extract(r'(-\d{3})')
    df['ccep'] = df_ccep
    df['cep_loc'] = df_cep
    df['cep_compl'] = df_cep_compl
    df_rplc = df['cep_compl'].str.replace('-', '')
    df['ccep_compl'] = df_rplc
    columns = ['cep', 'cep_compl']
    df.drop(columns, inplace=True, axis=1)
    df['ccep'].astype(int)
    df['cep_loc'].astype(int)
    df['ccep_compl'].astype(int)
    
    
    os.system(f'cp {caminho_arquivo} {bckp_path}')
    print(f'file copy: {file_bckp}')
    display(df)
                           
                    
                        
                
                

True
file extension csv or txt, ok! extension: .txt
file copy: /workspaces/api-geral/GEO/CEP/dados/landing/resultado_concat_20240513_162447.txt


,logradouro,bairro,localidade,uf,ibge,gia,ddd,siafi,ccep,cep_loc,ccep_compl
0,Rua Mairiporã,Mantiqueira,Mairiporã,SP,3528502,4339,11,6671,07613580,07613,580
1,Rua JasminsJd Mairipora I,Rio Abaixo,Mairiporã,SP,3528502,4339,11,6671,07605020,07605,020
2,Rua OrquideasJd Mairipora I,Rio Abaixo,Mairiporã,SP,3528502,4339,11,6671,07605065,07605,065
3,Rua GirassolJd Mairipora II,Rio Abaixo,Mairiporã,SP,3528502,4339,11,6671,07605225,07605,225
4,Rua das RosasJd Mairipora II,Rio Abaixo,Mairiporã,SP,3528502,4339,11,6671,07605200,07605,200
...,...,...,...,...,...,...,...,...,...,...,...
211,Avenida das Carnaubas,Olho D'Água,Mairiporã,SP,3528502,4339,11,6671,07601170,07601,170
212,Avenida das Camélias,Roseira,Mairiporã,SP,3528502,4339,11,6671,07619228,07619,228
213,Avenida dos Coqueiros,Barreiro,Mairiporã,SP,3528502,4339,11,6671,07611400,07611,400
214,Avenida Manoel Martins,Samambaia,Mairiporã,SP,3528502,4339,11,6671,07623025,07623,025


In [38]:
import json 
import os
from datetime import datetime
import os.path

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

bckp_path = '/workspaces/api-geral/GEO/CEP/ingestion/dados_bckp/'
layout = '/workspaces/api-geral/GEO/CEP/ingestion/dados_ing/layout/'
word_key = 'resultado_cep'


ultimo_timestamp = None
caminho_arquivo = None


for root, dirs, files in os.walk(layout):
    for nome_arquivo in files:
        if word_key in nome_arquivo:
            caminho_completo = os.path.join(root, nome_arquivo)
            timestamp = os.path.getmtime(caminho_completo)
            if ultimo_timestamp is None or timestamp > ultimo_timestamp:
                ultimo_timestamp = timestamp
                caminho_arquivo = caminho_completo
                checking = os.path.isfile(caminho_arquivo)
                if checking == True:
                    print(checking)
                else:
                    print('Error checking')
                if os.path.isfile(caminho_arquivo):
                    file_extension = os.path.splitext(caminho_arquivo)[1]
                    if file_extension.lower() in {'.json'}:
                        print(f'file extension json, ok! extension: {file_extension}')
                    else:
                        print('extension json error, not found')
                    print(caminho_arquivo)



True
file extension json, ok! extension: .json
/workspaces/api-geral/GEO/CEP/ingestion/dados_ing/layout/resultado_cep.json


In [9]:
import json
import pandas as pd
import sqlite3
import os
from datetime import datetime
import os.path
import time

print('Iniciando ingestão!')
time.sleep(3)
file_col_json = '/workspaces/api-geral/GEO/CEP/ingestion/dados_ing/src/estrutura_cep.json'
file_tb_json = '/workspaces/api-geral/GEO/CEP/ingestion/dados_ing/layout/resultado_cep.json'
print('Carregando variáveis json')

time.sleep(3)
# Ler configurações das colunas
with open(f'{file_col_json}', 'r') as f:
    config_colunas = json.load(f)
    print('Lendo estrutura de coluna json')

time.sleep(3)
# Ler configurações da tabela
with open(f'{file_tb_json}', 'r') as f:
    config_tabela = json.load(f)
    print('Lendo estrutura tabela json')

print('Carregando variáveis do arquivo de ingestão')
time.sleep(3)
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
bckp_path = '/workspaces/api-geral/GEO/CEP/ingestion/dados_bckp/'
word_key = 'resultado_cep'

ultimo_timestamp = None
caminho_arquivo = None

print('Realizando iteração')
print('Buscando o último arquivo ingerido de acordo com o timestamp')
time.sleep(3)
for root, dirs, files in os.walk(layout):
    for nome_arquivo in files:
        if word_key in nome_arquivo:
            caminho_completo = os.path.join(root, nome_arquivo)
            timestamp = os.path.getmtime(caminho_completo)
            if ultimo_timestamp is None or timestamp > ultimo_timestamp:
                ultimo_timestamp = timestamp
                caminho_arquivo = caminho_completo
                print('Arquivo encontrado!')
                print('Realizando checking do arquivo.')
                time.sleep(3)
                checking = os.path.isfile(caminho_arquivo)
                if checking:
                    print(checking)
                else:
                    print('Error checking')
                if os.path.isfile(caminho_arquivo):
                    file_extension = os.path.splitext(caminho_arquivo)[1]
                    if file_extension.lower() in {'.txt', '.csv'}:
                        print(f'File extension {file_extension} is supported.')
                    else:
                        print('Extension not supported.')
                    print(caminho_arquivo)

print('Carregando arquivo...')
time.sleep(3)

# Carregar dados
arquivo_dados = caminho_arquivo
if arquivo_dados.endswith('.txt'):
    dados = pd.read_csv(arquivo_dados, delimiter='\t', header=None, names=[col['nome'] for col in config_colunas['colunas']])
elif arquivo_dados.endswith('.csv'):
    dados = pd.read_csv(arquivo_dados, header=None, names=[col['nome'] for col in config_colunas['colunas']])
else:
    print("Formato de arquivo não suportado:", arquivo_dados)

print('Convertendo tipos de dados')
time.sleep(3)
# Conversão de tipos de dados
for col in config_colunas['colunas']:
    if col['tipo'] == 'int':
        dados[col['nome']] = dados[col['nome']].astype(int)
    elif col['tipo'] == 'float':
        dados[col['nome']] = dados[col['nome']].astype(float)
    elif col['tipo'] == 'data':
        dados[col['nome']] = pd.to_datetime(dados[col['nome']], format=col.get('formato', None))

print('Conectando no banco de dados')
# Conectar ao banco de dados
conn = sqlite3.connect('banco_cep_teste.db')
cursor = conn.cursor()

# Inserir dados na tabela
for index, row in dados.iterrows():
    values = tuple(row[col['nome']] for col in config_colunas['colunas'])
    sql = f"INSERT INTO {config_tabela['tabela']} ({', '.join([col['nome'] for col in config_colunas['colunas']])}) VALUES ({', '.join(['?' for _ in config_colunas['colunas']])})"
    cursor.execute(sql, values)

# Confirmar as alterações e fechar a conexão
conn.commit()
conn.close()

print("Ingestão de dados concluída com sucesso!")


Iniciando ingestão!
Carregando variáveis json
Lendo estrutura de coluna json
Lendo estrutura tabela json
Carregando variáveis do arquivo de ingestão
Realizando iteração
Buscando o último arquivo ingerido de acordo com o timestamp
Arquivo encontrado!
Realizando checking do arquivo.
True
Extension not supported.
/workspaces/api-geral/GEO/CEP/ingestion/dados_ing/layout/resultado_cep.json
Carregando arquivo...
Formato de arquivo não suportado: /workspaces/api-geral/GEO/CEP/ingestion/dados_ing/layout/resultado_cep.json
Convertendo tipos de dados


NameError: name 'dados' is not defined

In [30]:
import json
import pandas as pd
import sqlite3
import os
from datetime import datetime
import os.path
import time

print('iniciando ingestão!')
time.sleep(3)
file_col_json = '/workspaces/api-geral/GEO/CEP/ingestion/dados_ing/src/estrutura_cep.json'
file_tb_json = '/workspaces/api-geral/GEO/CEP/ingestion/dados_ing/layout/resultado_cep.json'
print('carregando variaveis json')

time.sleep(3)
# Ler configurações das colunas
with open(f'{file_col_json}', 'r') as f:
    config_colunas = json.load(f)
    print('lendo estrutura de coluna json')

time.sleep(3)
# Ler configurações da tabela
with open(f'{file_tb_json}', 'r') as f:
    config_tabela = json.load(f)
    print('lendo estrutura tabela json')

print('carregando variaveis do arquivo de ingestão')
time.sleep(3)
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
bckp_path = '/workspaces/api-geral/GEO/CEP/ingestion/dados_bckp/'
landing = '/workspaces/api-geral/GEO/CEP/dados/landing/'
word_key = 'resultado'

ultimo_timestamp = None
caminho_arquivo = None

print('realizando iteração')
print('buscando o ultimo arquivo ingerido de acordo com o timestamp')
time.sleep(3)
for root, dirs, files in os.walk(landing):
    for nome_arquivo in files:
        if word_key in nome_arquivo:
            caminho_completo = os.path.join(root, nome_arquivo)
            timestamp = os.path.getmtime(caminho_completo)
            if ultimo_timestamp is None or timestamp > ultimo_timestamp:
                ultimo_timestamp = timestamp
                caminho_arquivo = caminho_completo
                print('arquivo encontrado!')
                print('realizando checking do arquivo.')
                time.sleep(3)
                checking = os.path.isfile(caminho_arquivo)
                if checking == True:
                    print(checking)
                else:
                    print('Error checking')
                if os.path.isfile(caminho_arquivo):
                    file_extension = os.path.splitext(caminho_arquivo)[1]
                    if file_extension.lower() in {'.json'}:
                        print(f'file extension json, ok! extension: {file_extension}')
                    else:
                        print('extension json error, not found')
                    print(caminho_arquivo)

print('carregando arquivo...')
time.sleep(3)

# Carregar dados
arquivo_dados = caminho_arquivo
if arquivo_dados.endswith('.txt'):
    dados = pd.read_csv(arquivo_dados, delimiter='\t', header=None, names=[col['nome'] for col in config_colunas['colunas']])
elif arquivo_dados.endswith('.csv'):
    dados = pd.read_csv(arquivo_dados, header=None, names=[col['nome'] for col in config_colunas['colunas']])
elif arquivo_dados.endswith('.bin'):
    # Lógica para carregar dados de um arquivo binário
    pass
else:
    raise ValueError("Formato de arquivo não suportado")

print('convertendo tipos de dados')
time.sleep(3)
# Conversão de tipos de dados
for col in config_colunas['colunas']:
    if col['tipo'] == 'int':
        dados[col['nome']] = dados[col['nome']].astype(int)
    elif col['tipo'] == 'float':
        dados[col['nome']] = dados[col['nome']].astype(float)
    elif col['tipo'] == 'data':
        dados[col['nome']] = pd.to_datetime(dados[col['nome']], format=col.get('formato', None))

print('conctando no banco de dados')
# Conectar ao banco de dados
banco = '/workspaces/api-geral/GEO/CEP/sql/db/banco_cep_teste.db'
conn = sqlite3.connect(f'{banco}')
cursor = conn.cursor()

# Criar tabela com base na estrutura definida no arquivo JSON
print('Criando tabela no banco de dados...')
sql_create_table = f"CREATE TABLE IF NOT EXISTS {config_tabela['tabela']} ("
for col in config_colunas['colunas']:
    sql_create_table += f"{col['nome']} {col['tipo']}, "
sql_create_table = sql_create_table[:-2] + ")"
cursor.execute(sql_create_table)

# Inserir dados na tabela
print('Inserindo dados na tabela...')
for index, row in dados.iterrows():
    mapped_values = []
    for col in config_colunas['colunas']:
        mapped_values.append(row.get(col['nome'], None))
    print("Valores mapeados:", mapped_values)
    sql = f"INSERT INTO {config_tabela['tabela']} ({', '.join([col['nome'] for col in config_colunas['colunas']])}) VALUES ({', '.join(['?' for _ in mapped_values])})"
    print("SQL:", sql)
    cursor.execute(sql, mapped_values)

# Confirmar as alterações e fechar a conexão
print('Commit e fechamento da conexão...')
conn.commit()
conn.close()

print("Ingestão de dados concluída com sucesso!")


iniciando ingestão!


carregando variaveis json
lendo estrutura de coluna json
lendo estrutura tabela json
carregando variaveis do arquivo de ingestão
realizando iteração
buscando o ultimo arquivo ingerido de acordo com o timestamp
arquivo encontrado!
realizando checking do arquivo.
True
extension json error, not found
/workspaces/api-geral/GEO/CEP/dados/landing/resultado_concat_20240513_162447.txt
carregando arquivo...
convertendo tipos de dados
conctando no banco de dados
Criando tabela no banco de dados...
Inserindo dados na tabela...
Valores mapeados: ['cep;logradouro;bairro;localidade;uf;ibge;gia;ddd;siafi', nan, nan, nan, nan, nan, nan, nan]
SQL: INSERT INTO tb_cep_brnz (logradouro, bairro, localidade, uf, ibge, gia, ddd, siafi) VALUES (?, ?, ?, ?, ?, ?, ?, ?)
Valores mapeados: ['07613-580;Rua Mairiporã;Mantiqueira;Mairiporã;SP;3528502;4339;11;6671', nan, nan, nan, nan, nan, nan, nan]
SQL: INSERT INTO tb_cep_brnz (logradouro, bairro, localidade, uf, ibge, gia, ddd, siafi) VALUES (?, ?, ?, ?, ?, ?, ?, 

In [31]:
# Conectar ao banco de dados
banco = '/workspaces/api-geral/GEO/CEP/sql/db/banco_cep_teste.db'
conn = sqlite3.connect(f'{banco}')
cursor = conn.cursor()

tabela = 'tb_cep_brnz'
query = f'''
select * from {tabela}
'''
cursor.execute(query)
resultados = cursor.fetchall()
print('Resultado:')
for linha in resultados:
    print(linha)    

Resultado:
('cep;logradouro;bairro;localidade;uf;ibge;gia;ddd;siafi', None, None, None, None, None, None, None)
('07613-580;Rua Mairiporã;Mantiqueira;Mairiporã;SP;3528502;4339;11;6671', None, None, None, None, None, None, None)
('07605-020;Rua JasminsJd Mairipora I;Rio Abaixo;Mairiporã;SP;3528502;4339;11;6671', None, None, None, None, None, None, None)
('07605-065;Rua OrquideasJd Mairipora I;Rio Abaixo;Mairiporã;SP;3528502;4339;11;6671', None, None, None, None, None, None, None)
('07605-225;Rua GirassolJd Mairipora II;Rio Abaixo;Mairiporã;SP;3528502;4339;11;6671', None, None, None, None, None, None, None)
('07605-200;Rua das RosasJd Mairipora II;Rio Abaixo;Mairiporã;SP;3528502;4339;11;6671', None, None, None, None, None, None, None)
('07605-230;Rua AntúriosJd Mairipora II;Rio Abaixo;Mairiporã;SP;3528502;4339;11;6671', None, None, None, None, None, None, None)
('07605-070;Rua dos CravosJd Mairipora I;Rio Abaixo;Mairiporã;SP;3528502;4339;11;6671', None, None, None, None, None, None, None

In [32]:
query = 'drop table tb_cep_brnz'
cursor.execute(query)
print('tabela excluida')
conn.commit()
conn.close()

tabela excluida


In [7]:
import json 

layout = '/workspaces/api-geral/GEO/CEP/ingestion/dados_ing/layout/resultado_cep.json'

read_layout = json.load(str(layout))
print(read_layout)

AttributeError: 'str' object has no attribute 'read'

In [ ]:
import os

file_path = "C:/folder/file.mp3"
if os.path.isfile(file_path):
    file_extension = os.path.splitext(file_path)[1]
    if file_extension.lower() in {'.mp3', '.flac', '.ogg'}:
        print("It's a music file")
    elif file_extension.lower() in {'.jpg', '.jpeg', '.png'}:
        print("It's an image file")

In [ ]:
                for check in caminho_arquivo:
                    ext = os.path.splitext(check)[-2].lower()
                    if ext == '.txt':
                        print(f'extension: {check}')
                    elif ext == '.csv':
                        print(f'extension: {check}')
                    else:
                        print(f'error extension format.')

In [ ]:
filepaths = ["/folder/soundfile.mp3", "folder1/folder/soundfile.flac"]

for fp in filepaths:
    # Split the extension from the path and normalise it to lowercase.
    ext = os.path.splitext(fp)[-1].lower()

    # Now we can simply use == to check for equality, no need for wildcards.
    if ext == ".mp3":
        print fp, "is an mp3!"
    elif ext == ".flac":
        print fp, "is a flac file!"
    else:
        print fp, "is an unknown file format."